In [1]:
import synapseclient
import pandas as pd
import os
import wget
from synapseclient import Project, Folder, File, Link
import requests
import numpy as np
# from synapseclient import Activity
# from synapseclient import Entity, Project, Folder, File, Link
# from synapseclient import Evaluation, Submission, SubmissionStatus
# from synapseclient import Wiki


In [2]:
syn = synapseclient.Synapse()
PAT = "ENTER YOUR PERSONAL ACCESS TOKEN HERE"


syn.login(authToken=PAT)


UPGRADE AVAILABLE

A more recent version of the Synapse Client (3.0.0) is available. Your version (2.7.2) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 3.0.0 release notes

https://python-docs.synapse.org/build/html/news.html



Welcome, JJacobson95!



In [3]:
entity = syn.get('syn51674470', downloadLocation='.')
entity = syn.get('syn25714248', downloadLocation='.')
entity = syn.get('syn32533104', downloadLocation='.')
entity = syn.get('syn32529921', downloadLocation='.')
entity = syn.get('syn26642974', downloadLocation='.')
entity = syn.get('syn26427390', downloadLocation='.')

#patient data
entity = syn.get('syn26642974', downloadLocation='.')

In [4]:

def download_from_github(raw_url, save_path):
    response = requests.get(raw_url)

    with open(save_path, 'wb') as f:
        f.write(response.content)
    return

In [5]:

def retrieve_figshare_data(url):
    """
    Retrieve data from a given figshare URL.
    
    Return: File name (genes.csv)
    """
    files_0 = os.listdir()
    wget.download(url)
    files_1 = os.listdir()
    new_file = str(next(iter(set(files_1) - set(files_0))))
    return new_file



In [6]:
def map_and_combine(df, data_type, entrez_map_file, improve_map_file, sample_map=None):
    """
    Maps and combines dataframes based on their data_type. It then merges 
    the final dataframe with provided metadata.
    """
    # Initialize the list to hold mapped dataframes

    # Load mapping files
    samples = sample_map
    genes = pd.read_csv(entrez_map_file)          # Map gene_name to entrez_id
    
    improve = pd.read_csv(improve_map_file)        # Map sample_id to improve_sample_id

    # Process each dataframe based on its data_type
#     for df in dataframe_list:
    if data_type == "transcriptomics":
        mapped_df = df.merge(genes, left_on='Gene', right_on='gene_symbol', how='left').reindex(
                        columns=['transcriptomics', 'entrez_id', "sample_id"])
#         mapped_df.drop(columns=['gene_id', 'gene_name', 'gene_type'], inplace=True)


    elif data_type == "mutations":
#         mapped_df = df.reindex(columns=['Entrez_Gene_Id', 'symbol', 'hgvsc'])
        mapped_df = df.merge(genes, left_on='symbol', right_on='gene_symbol', how='left').reindex(
                        columns=['hgvsc', 'entrez_id', "alt_ID"])
        mapped_df.rename(columns={"hgvsc": "mutations"}, inplace=True)
        mapped_df.rename(columns={"alt_ID": "sample_id"}, inplace=True)
        mapped_df.rename(columns={"Entrez_Gene_Id": "entrez_id"}, inplace=True)

    elif data_type == "proteomics":

        # Update 'sampleID' in mapped_ids dataframe
        sample_map['sampleID'] = sample_map['sampleID'].str.split('_').apply(lambda x: x[2])

        # Rename 'sampleID' to 'id'
        sample_map.rename(columns={"sampleID": "id"}, inplace=True)
#         print("sample_map", sample_map)
        # Reindex the columns of mapped_ids dataframe
        sample_map = mapped_ids.reindex(columns=['labId', "id"])        
        
        # Merge p_df with mapped_ids
        df = df.merge(mapped_ids, left_on='id', right_on='id', how='left')

        # Reindex the columns of p_df
        df = df.reindex(columns=["Protein", "proteomics", "labId"])

        # Merge df with genes and reindex + rename columns
        mapped_df = df.merge(genes, left_on='Protein', right_on='gene_symbol', how='left').reindex(
            columns=['proteomics', 'entrez_id', 'labId'])

        mapped_df.rename(
            columns={
                "labId": "sample_id",
            },
            inplace=True
        )



    # Add improve ID then Drop the sample_id and other_id columns
    
    mapped_df = pd.merge(mapped_df, improve[['other_id', 'improve_sample_id']], left_on='sample_id', right_on='other_id', how='left')
    mapped_df.drop(columns=['sample_id', 'other_id'], inplace=True)
    mapped_df.insert(0, 'improve_sample_id', mapped_df.pop('improve_sample_id'))
    mapped_df['source'] = 'synapse'
    mapped_df['study'] = 'BeatAML'

    # Concatenate the list of dataframes into one final dataframe
    final_dataframe = mapped_df


    return final_dataframe


In [7]:
gene_url = "https://figshare.com/ndownloader/files/40576109?private_link=525f7777039f4610ef47"

entrez_map_file = retrieve_figshare_data(gene_url)

100% [..........................................................................] 8156240 / 8156240

In [8]:
samples_url = "https://figshare.com/ndownloader/files/42289053?private_link=f05259d19b8c34a9a53d"

improve_map_file = retrieve_figshare_data(samples_url)

100% [..............................................................................] 23019 / 23019

In [9]:
#Transcriptomics
#entrez_id
#transcriptomics
#improve_sample_id
#source
#study

t_df = pd.read_csv("BeatAML_Waves1to4_RNA_data_normalized.txt", sep = '\t')

t_df = t_df.reset_index().rename(columns={'index': 'Gene'})

t_df = pd.melt(t_df, id_vars=['Gene'], var_name='sample_id', value_name='transcriptomics')

# t_df

t_df = map_and_combine(t_df, "transcriptomics", entrez_map_file, improve_map_file)




In [10]:
t_df
t_df.to_csv("transcriptomics.csv")

In [11]:
#Proteomics
#entrez_id
#proteomics
#improve_sample_id
#source
#study



#How to read in samples file
# df = pd.read_csv("PNNL_clinical_summary_12_08_2021_updated_OS_4patients_02_28_2022.txt", 
#                  sep='\s+', 
#                  encoding='mac-roman',  
#                  quotechar='"',
#                  error_bad_lines=False,
#                  warn_bad_lines=True,
#                  index_col=None)




p_df = pd.read_csv("ptrc_ex10_crosstab_global_gene_corrected.txt", sep = '\t')

p_df = p_df.reset_index().rename(columns={'index': 'Protein'})

p_df = pd.melt(p_df, id_vars=['Protein'], var_name='id', value_name='proteomics')

p_df

# mapped_ids = pd.read_csv("Data Available for Proteomic Samples.xlsx")
mapped_ids = pd.read_excel("Data Available for Proteomic Samples.xlsx")

p_df = map_and_combine(p_df, "proteomics", entrez_map_file, improve_map_file, mapped_ids)



In [12]:
p_df
p_df.to_csv("proteomics.csv")

In [13]:
#Mutations
#entrez_id
#mutation
#improve_sample_id
#source
#study


m_df = pd.read_csv("beataml_wes_wv1to4_mutations_177_samples.txt", sep = '\t')
m_df = map_and_combine(m_df, "mutations", entrez_map_file, improve_map_file)


In [14]:
m_df
m_df.to_csv("mutations.csv")

In [15]:

#Note - This file was converted from txt to csv due to byte reading issues.
#Note - assign_improve_ids.py was run on this prior to here.
# summary = pd.read_csv("PNNL_clinical_summary_12_08_2021_updated_OS_4patients_02_28_2022.csv")

# samples = summary[["labId","improve_sample_id","specificDxAtInclusion","specimenType"]]

In [16]:
# samples = summary[["labId","improve_sample_id","specificDxAtInclusion","specimenType"]]
# samples.rename(columns={"labId": "other_id"}, inplace=True)
# samples.rename(columns={"specificDxAtInclusion": "other_names"}, inplace=True)
# samples.rename(columns={"specimenType": "common_name"}, inplace=True)


# samples["cancer_type"] = "ACUTE MYELOID LEUKAEMIA"

# samples["model_type"] = "ex vivo"
# samples["other_id_source"] = "beatAML"

In [17]:
# samples

In [18]:
# samples.to_csv("samples.csv", index = False)


In [19]:

# def retreive_drug_info(compound_name):
#     url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound_name}/property/CanonicalSMILES,IsomericSMILES,InChIKey,MolecularFormula,MolecularWeight/JSON"

#     response = requests.get(url)
#     data = response.json()

#     properties = data["PropertyTable"]["Properties"][0]
#     canSMILES = properties["CanonicalSMILES"]
#     isoSMILES = properties["IsomericSMILES"]
#     inchikey = properties["InChIKey"]
#     formula = properties["MolecularFormula"]
#     weight = properties["MolecularWeight"]

#     return(canSMILES, isoSMILES, inchikey, formula, weight)

In [20]:
def retrieve_drug_info(compound_name):
    if pd.isna(compound_name):
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound_name}/property/CanonicalSMILES,IsomericSMILES,InChIKey,MolecularFormula,MolecularWeight/JSON"
    response = requests.get(url)

    if response.status_code != 200:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    data = response.json()
    if "PropertyTable" in data:
        properties = data["PropertyTable"]["Properties"][0]
        canSMILES = properties.get("CanonicalSMILES", np.nan)
        isoSMILES = properties.get("IsomericSMILES", np.nan)
        inchikey = properties.get("InChIKey", np.nan)
        formula = properties.get("MolecularFormula", np.nan)
        weight = properties.get("MolecularWeight", np.nan)

        return canSMILES, isoSMILES, inchikey, formula, weight
    else:
        return np.nan, np.nan, np.nan, np.nan, np.nan

In [21]:

def update_dataframe_with_pubchem(d_df):
    # Get unique chem_name values and retrieve their data
    chem_names = d_df['chem_name'].dropna().unique()
    chem_data_dict = {}
    for name in chem_names:
        print("Attempting to call pubchem API for chem_name: ", name)
        chem_data_dict[name] = retrieve_drug_info(name)

    # Identify rows whose chem_name failed and retrieve the other_name for those rows
    failed_chem_names = {k for k, v in chem_data_dict.items() if all(pd.isna(val) for val in v)}
    other_names = d_df[d_df['chem_name'].isin(failed_chem_names)]['other_name'].dropna().unique()
    other_data_dict = {}
    for name in other_names:
        print("Attempting to call pubchem API for other_name: ", name)
        other_data_dict[name] = retrieve_drug_info(name)

    # Combine both dictionaries for easy lookup
    data_dict = {**chem_data_dict, **other_data_dict}

    # Update the DataFrame using the data dictionary
    for idx, row in d_df.iterrows():
        if row['chem_name'] in data_dict and not all(pd.isna(val) for val in data_dict[row['chem_name']]):
            values = data_dict[row['chem_name']]
        else:
            values = data_dict.get(row['other_name'], (np.nan, np.nan, np.nan, np.nan, np.nan))
        
        d_df.at[idx, "canSMILES"] = values[0]
        d_df.at[idx, "isoSMILES"] = values[1]
        d_df.at[idx, "inchikey"] = values[2]
        d_df.at[idx, "formula"] = values[3]
        d_df.at[idx, "weight"] = values[4]
    
    return d_df

In [22]:
def merge_drug_info(d_df,drug_map):
    result_df = d_df.merge(drug_map[['isoSMILES', 'improve_drug_id']], on='isoSMILES', how='left')
    result_df = result_df[["sample_id","improve_drug_id","chem_name", "auc","formula","weight", "canSMILES","inchikey","isoSMILES"]]
    return result_df


In [23]:
# Drug map file
# drug_url = retrieve_figshare_data("https://figshare.com/ndownloader/files/41259273?private_link=525f7777039f4610ef47")
def format_drug_map(drug_map_path):
    drug_map = pd.read_csv(drug_map_path, sep = "\t")
    drug_map = drug_map.drop_duplicates(subset='isoSMILES', keep='first')
    return drug_map

In [24]:
#Drug Response
def format_drug_df(drug_path):
    d_df = pd.read_csv(drug_path, index_col=None)
    d_df.drop("Unnamed: 0",axis='columns',inplace=True)
    d_df[['chem_name', 'other_name']] = d_df['inhibitor'].str.extract(r'^(.*?)\s*(?:\((.+)\))?$')
    d_df["chem_name"] = d_df["chem_name"].str.replace('\s-\s', ':')
    return d_df

In [25]:
def add_improve_id(previous_df, new_df):
    
    # Extract the maximum value of improve_drug_id from the previous dataframe
    # Assuming the format is SMI_number. We get rid of 'SMI_' and then convert to int.
    max_id = max([int(val.replace('SMI_', '')) for val in previous_df['improve_drug_id'].tolist() if pd.notnull(val) and val.startswith('SMI_')])

    # Identify isoSMILES in the new dataframe that don't exist in the old dataframe
    unique_new_smiles = set(new_df['isoSMILES']) - set(previous_df['isoSMILES'])
    
    # Identify rows in the new dataframe with isoSMILES that are unique and where improve_drug_id is NaN
    mask = (new_df['isoSMILES'].isin(unique_new_smiles)) & (new_df['improve_drug_id'].isna())
    
    # Create a mapping of unique new isoSMILES to improve_drug_id
    id_map = {}
    for smiles in unique_new_smiles:
        max_id += 1
        id_map[smiles] = f"SMI_{max_id}"
    
    # Apply the mapping to the new dataframe for rows with unique isoSMILES and NaN improve_drug_id
    new_df.loc[mask, 'improve_drug_id'] = new_df['isoSMILES'].map(id_map)
    
    return new_df

In [26]:
def map_exp_to_improve(df,improve_map_file):
    improve = pd.read_csv(improve_map_file)        # Map sample_id to improve_sample_id
    mapped_df = pd.merge(df, improve[['other_id', 'improve_sample_id']], left_on='sample_id', right_on='other_id', how='left')
    mapped_df.drop(columns=['sample_id', 'other_id'], inplace=True)
    mapped_df.insert(0, 'improve_sample_id', mapped_df.pop('improve_sample_id'))
    mapped_df['source'] = 'synapse'
    mapped_df['study'] = 'BeatAML'
    mapped_df['ic50'] = np.nan
    mapped_df['ec50'] = np.nan
    mapped_df['ec50se'] = np.nan
    mapped_df['einf'] = np.nan
    mapped_df['hs'] = np.nan
    mapped_df['aac1'] = np.nan
    mapped_df['auc1'] = np.nan
    mapped_df['dss1'] = np.nan
    return mapped_df

In [27]:
def remove_sixth_from_last_quote(s):
    reversed_s = s[::-1]  # Reverse the string
    count = 0
    index_to_remove = None
    for i, char in enumerate(reversed_s):
        if char == '"':
            count += 1
        if count == 6:
            index_to_remove = i
            break
    # Remove the 6th occurrence of the quote character
    if index_to_remove is not None:
        reversed_s = reversed_s[:index_to_remove] + reversed_s[index_to_remove+1:]
    return reversed_s[::-1]  # Reverse again to obtain the original order

In [28]:
def modify_patient_file(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        lines = f.readlines()
    # Modify the 110th line (0-based index is 109)
    lines[109] = remove_sixth_from_last_quote(lines[109])
    with open(file_path, 'w', encoding='utf-8') as f:
        f.writelines(lines)

In [ ]:
def generate_samples_file()
    file_path = 'PNNL_clinical_summary_12_08_2021_updated_OS_4patients_02_28_2022.txt'
    modify_patient_file(file_path)
    samples = pd.read_csv('PNNL_clinical_summary_12_08_2021_updated_OS_4patients_02_28_2022.txt', delimiter=' ', skipinitialspace=True, encoding='ISO-8859-1', quotechar='"')
    maxval = max(pd.read_csv('https://raw.githubusercontent.com/PNNL-CompBio/candleDataProcessing/main/hcmi/samples.csv').improve_sample_id)
    # Create a dictionary that maps unique lab-id values to sequential integers
    mapping = {labId: i for i, labId in enumerate(samples['labId'].unique(), start=(int(maxval)+1))}
    # Use the map method to create the new column based on the lab-id column
    samples['improve_sample_id'] = samples['labId'].map(mapping)
    samples = samples[["labId","improve_sample_id","specificDxAtInclusion","specimenType"]]
    samples.rename(columns={"labId": "other_id"}, inplace=True)
    samples.rename(columns={"specificDxAtInclusion": "other_names"}, inplace=True)
    samples.rename(columns={"specimenType": "common_name"}, inplace=True)
    samples["cancer_type"] = "ACUTE MYELOID LEUKAEMIA"
    samples["model_type"] = "ex vivo"
    samples["other_id_source"] = "beatAML"
    samples.to_csv("samples.csv", index = False)

In [ ]:
generate_samples_file

In [30]:
drug_path = "drug_response.csv"
drug_map_path = "drugs_map.tsv.gz"
drug_url = "https://raw.githubusercontent.com/PNNL-CompBio/candleDataProcessing/main/cell_line/drugs_by_structure.tsv.gz"

download_from_github(drug_url, drug_map_path)
drug_map = format_drug_map(drug_map_path)
d_df = format_drug_df(drug_path)
d_df = update_dataframe_with_pubchem(d_df)
d_res = merge_drug_info(d_df, drug_map)
d_res = add_improve_id(drug_map, d_res)

#split df to drug
drug_res = d_res[["improve_drug_id","chem_name","formula","weight","inchikey","canSMILES","isoSMILES"]]
drug_res.rename(columns={"inchikey": "inCHIKey"}, inplace=True)

# drug_res.to_csv("drugs.tsv",sep="\t")

# split df to experiment
exp_res = d_res[["sample_id","improve_drug_id","auc"]]
exp_res = map_exp_to_improve(exp_res,improve_map_file)


[WARNING] /var/folders/wt/_yr7rg_13t76sq_q5cw57hxw0000gn/T/ipykernel_84914/2428304630.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  d_df["chem_name"] = d_df["chem_name"].str.replace('\s-\s', ':')



Attempting to call pubchem API for chem_name:  AKT Inhibitor IV
Attempting to call pubchem API for chem_name:  AKT Inhibitor X
Attempting to call pubchem API for chem_name:  AMPK Inhibitor
Attempting to call pubchem API for chem_name:  AST-487
Attempting to call pubchem API for chem_name:  AZD1152-HQPA
Attempting to call pubchem API for chem_name:  Alisertib
Attempting to call pubchem API for chem_name:  Bortezomib
Attempting to call pubchem API for chem_name:  CYT387
Attempting to call pubchem API for chem_name:  Canertinib
Attempting to call pubchem API for chem_name:  Crenolanib
Attempting to call pubchem API for chem_name:  Doramapimod
Attempting to call pubchem API for chem_name:  Dovitinib
Attempting to call pubchem API for chem_name:  Erlotinib
Attempting to call pubchem API for chem_name:  Flavopiridol
Attempting to call pubchem API for chem_name:  GDC-0941
Attempting to call pubchem API for chem_name:  GW-2580
Attempting to call pubchem API for chem_name:  Gefitinib
Attempting

Attempting to call pubchem API for chem_name:  Doramapimod:GW-2580
Attempting to call pubchem API for chem_name:  Doramapimod:Idelalisib
Attempting to call pubchem API for chem_name:  Doramapimod:Midostaurin
Attempting to call pubchem API for chem_name:  Doramapimod:NF-kb ActInh
Attempting to call pubchem API for chem_name:  Doramapimod:Ruxolitinib
Attempting to call pubchem API for chem_name:  Doramapimod:Sorafenib
Attempting to call pubchem API for chem_name:  GW-2580:Doramapimod
Attempting to call pubchem API for chem_name:  GW-2580:KI20227
Attempting to call pubchem API for chem_name:  GW-2580:Midostaurin
Attempting to call pubchem API for chem_name:  GW-2580:Pazopanib
Attempting to call pubchem API for chem_name:  GW-2580:Sunitinib
Attempting to call pubchem API for chem_name:  Idelalisib:Dasatinib
Attempting to call pubchem API for chem_name:  Idelalisib:GW-2580
Attempting to call pubchem API for chem_name:  Idelalisib:Midostaurin
Attempting to call pubchem API for chem_name:  Id

Attempting to call pubchem API for chem_name:  GSK-2879552
Attempting to call pubchem API for chem_name:  Azacytidine:Venetoclax
Attempting to call pubchem API for chem_name:  Bortezomib:AT-101
Attempting to call pubchem API for chem_name:  Bortezomib:Birinapant
Attempting to call pubchem API for chem_name:  Bortezomib:GSK2879552
Attempting to call pubchem API for chem_name:  Bortezomib:OTX-015
Attempting to call pubchem API for chem_name:  Bortezomib:Palbociclib
Attempting to call pubchem API for chem_name:  Bortezomib:Venetoclax
Attempting to call pubchem API for chem_name:  Doramapimod:AT-101
Attempting to call pubchem API for chem_name:  Doramapimod:Birinapant
Attempting to call pubchem API for chem_name:  Doramapimod:GSK2879552
Attempting to call pubchem API for chem_name:  Doramapimod:OTX-015
Attempting to call pubchem API for chem_name:  GW-2580:AT-101
Attempting to call pubchem API for chem_name:  GW-2580:Birinapant
Attempting to call pubchem API for chem_name:  GW-2580:GSK2879

[WARNING] /var/folders/wt/_yr7rg_13t76sq_q5cw57hxw0000gn/T/ipykernel_84914/3316055387.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drug_res.rename(columns={"inchikey": "inCHIKey"}, inplace=True)



In [31]:
drug_res

,improve_drug_id,chem_name,formula,weight,inCHIKey,canSMILES,isoSMILES
0,SMI_56919,AKT Inhibitor IV,C31H27IN4S,614.5,NAYRELMNTQSBIN-UHFFFAOYSA-M,CC[N+]1=C(N(C2=C1C=C(C=C2)C3=NC4=CC=CC=C4S3)C5...,CC[N+]1=C(N(C2=C1C=C(C=C2)C3=NC4=CC=CC=C4S3)C5...
1,SMI_56921,AKT Inhibitor X,C20H26Cl2N2O,381.3,SVKSJUIYYCQZEC-UHFFFAOYSA-N,CCN(CC)CCCCN1C2=CC=CC=C2OC3=C1C=C(C=C3)Cl.Cl,CCN(CC)CCCCN1C2=CC=CC=C2OC3=C1C=C(C=C3)Cl.Cl
2,SMI_56939,AMPK Inhibitor,C24H25N5O,399.5,XHBVYDAKJHETMP-UHFFFAOYSA-N,C1CCN(CC1)CCOC2=CC=C(C=C2)C3=CN4C(=C(C=N4)C5=C...,C1CCN(CC1)CCOC2=CC=C(C=C2)C3=CN4C(=C(C=N4)C5=C...
3,SMI_56922,AST-487,C26H30F3N7O2,529.6,ODPGGGTTYSGTGO-UHFFFAOYSA-N,CCN1CCN(CC1)CC2=C(C=C(C=C2)NC(=O)NC3=CC=C(C=C3...,CCN1CCN(CC1)CC2=C(C=C(C=C2)NC(=O)NC3=CC=C(C=C3...
4,SMI_49320,AZD1152-HQPA,C26H30FN7O3,507.6,QYZOGCMHVIGURT-UHFFFAOYSA-N,CCN(CCCOC1=CC2=C(C=C1)C(=NC=N2)NC3=NNC(=C3)CC(...,CCN(CCCOC1=CC2=C(C=C1)C(=NC=N2)NC3=NNC(=C3)CC(...
...,...,...,...,...,...,...,...
27109,SMI_48842,Vismodegib,C19H14Cl2N2O3S,421.3,BPQMGSKTAYIVFO-UHFFFAOYSA-N,CS(=O)(=O)C1=CC(=C(C=C1)C(=O)NC2=CC(=C(C=C2)Cl...,CS(=O)(=O)C1=CC(=C(C=C1)C(=O)NC2=CC(=C(C=C2)Cl...
27110,SMI_49301,Volasertib,C34H50N8O3,618.8,SXNJFOWDRLKDSF-XKHVUIRMSA-N,CCC1C(=O)N(C2=CN=C(N=C2N1C(C)C)NC3=C(C=C(C=C3)...,CC[C@@H]1C(=O)N(C2=CN=C(N=C2N1C(C)C)NC3=C(C=C(...
27111,SMI_48825,XAV-939,C14H11F3N2OS,312.31,KLGQSVMIPOVQAX-UHFFFAOYSA-N,C1CSCC2=C1N=C(NC2=O)C3=CC=C(C=C3)C(F)(F)F,C1CSCC2=C1N=C(NC2=O)C3=CC=C(C=C3)C(F)(F)F
27112,SMI_56928,XMD 8-87,C24H27N7O2,445.5,LGLHCXISMKHLIK-UHFFFAOYSA-N,CN1CCN(CC1)C2=CC(=C(C=C2)NC3=NC=C4C(=N3)N(C5=C...,CN1CCN(CC1)C2=CC(=C(C=C2)NC3=NC=C4C(=N3)N(C5=C...


In [32]:
exp_res

,improve_sample_id,improve_drug_id,auc,source,study,ic50,ec50,ec50se,einf,hs,aac1,auc1,dss1
0,111374.0,SMI_56919,109.414835,synapse,BeatAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,111374.0,SMI_56921,183.984149,synapse,BeatAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111374.0,SMI_56939,183.478483,synapse,BeatAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111374.0,SMI_56922,242.784602,synapse,BeatAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,111374.0,SMI_49320,188.768480,synapse,BeatAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27109,111583.0,SMI_48842,243.001338,synapse,BeatAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27110,111583.0,SMI_49301,156.133267,synapse,BeatAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27111,111583.0,SMI_48825,252.977539,synapse,BeatAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27112,111583.0,SMI_56928,165.025345,synapse,BeatAML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Experiment Data
# auc
# improve_sample_id
# improve drug id
# source
# study
# empty:
# ic50
# ec50
# ec50se
# einf
# hs
# aac1
# auc1
# dss1




In [34]:
# # All Non mapped drugs
# na_chem_names = result_df[result_df['improve_drug_id'].isna()]['chem_name'].unique().tolist()
# print(len(na_chem_names))
# print(na_chem_names)

In [35]:
# # Non mapped drugs excluding interactions
# print(len([item for item in na_chem_names if ':' not in item]))
# print([item for item in na_chem_names if ':' not in item])

In [36]:
# # All Non mapped drugs - better view
# pd.set_option('display.max_rows', 320)
# pd.set_option('display.min_rows', 320)
# result_df[["chem_name", "isoSMILES","improve_drug_id"]].drop_duplicates()


In [37]:
# pd.set_option('display.max_rows', 20)
# pd.set_option('display.min_rows', 20)

In [38]:
#PNNL_clinical_summary_12_08_2021_updated_OS_4patients_02_28_2022.txt is messy
#This is needed to remove a random quote from the 110th line. And to fix the encoding. 

# def remove_sixth_from_last_quote(s):
#     reversed_s = s[::-1]  # Reverse the string
#     count = 0
#     index_to_remove = None

#     for i, char in enumerate(reversed_s):
#         if char == '"':
#             count += 1
#         if count == 6:
#             index_to_remove = i
#             break

#     # Remove the 6th occurrence of the quote character
#     if index_to_remove is not None:
#         reversed_s = reversed_s[:index_to_remove] + reversed_s[index_to_remove+1:]

#     return reversed_s[::-1]  # Reverse again to obtain the original order

# def modify_file(file_path):
#     with open(file_path, 'r', encoding='ISO-8859-1') as f:
#         lines = f.readlines()

#     # Modify the 110th line (0-based index is 109)
#     lines[109] = remove_sixth_from_last_quote(lines[109])

#     with open(file_path, 'w', encoding='utf-8') as f:
#         f.writelines(lines)

# file_path = 'PNNL_clinical_summary_12_08_2021_updated_OS_4patients_02_28_2022.txt'
# modify_file(file_path)


In [39]:
# samples